# 3. Model Training and Evaluation

This notebook covers:
- Training LSTM + XGBoost hybrid model
- Hyperparameter tuning
- Model evaluation and metrics
- Training volatility prediction model


In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

project_root = Path('.').resolve().parent
sys.path.insert(0, str(project_root))

from data.fetch_data import CryptoDataFetcher
from src.zigzag_indicator import ZigZagIndicator
from src.features import FeatureEngineer
from src.models import LSTMXGBoostModel, VolatilityModel
from src.utils import time_series_split, normalize_data

## Step 1: Prepare Data

In [ ]:
fetcher = CryptoDataFetcher()
btc_15m = fetcher.fetch_symbol_timeframe('BTCUSDT', '15m')

zigzag = ZigZagIndicator(depth=12, deviation=5, backstep=2)
btc_15m = zigzag.label_kbars(btc_15m)

fe = FeatureEngineer(lookback_periods=[5, 10, 20, 50, 200])
btc_15m = fe.calculate_all_features(btc_15m)

feature_cols = fe.get_feature_columns(btc_15m)
btc_15m[feature_cols] = btc_15m[feature_cols].fillna(method='ffill').fillna(0)

print(f'Data prepared. Shape: {btc_15m.shape}, Features: {len(feature_cols)}')

## Step 2: Time Series Split

In [ ]:
train_df, val_df, test_df = time_series_split(btc_15m, train_ratio=0.7, validation_ratio=0.15)
print(f'Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}')

## Step 3: Prepare LSTM Sequences

In [ ]:
# Remove non-numeric columns
numeric_feature_cols = [col for col in feature_cols if col not in ['symbol', 'timestamp', 'date']]

print(f'Total features: {len(feature_cols)}')
print(f'Numeric features (after filtering): {len(numeric_feature_cols)}')

model = LSTMXGBoostModel(timesteps=60, n_features=len(numeric_feature_cols))

X_train = train_df[numeric_feature_cols].values
y_train = train_df['zigzag_label'].values
X_val = val_df[numeric_feature_cols].values
y_val = val_df['zigzag_label'].values
X_test = test_df[numeric_feature_cols].values
y_test = test_df['zigzag_label'].values

# Normalize
X_train_norm, train_params = normalize_data(X_train)
X_val_norm = (X_val - train_params['mean']) / (train_params['std'] + 1e-8)
X_test_norm = (X_test - train_params['mean']) / (train_params['std'] + 1e-8)

# Create sequences
X_train_seq, y_train_seq = model.create_sequences(X_train_norm, y_train)
X_val_seq, y_val_seq = model.create_sequences(X_val_norm, y_val)
X_test_seq, y_test_seq = model.create_sequences(X_test_norm, y_test)

print(f'Sequences created.')
print(f'Train: {X_train_seq.shape}, Val: {X_val_seq.shape}, Test: {X_test_seq.shape}')

## Step 4: Train Model (This will take 10-30 minutes)

In [ ]:
print('Starting model training...')
history = model.train(
    X_train_seq, y_train_seq,
    X_val_seq, y_val_seq,
    epochs=100,
    batch_size=32,
    early_stopping_patience=10
)

print(f'Training complete!')
print(f'LSTM Train Accuracy: {history["lstm_train_acc"]:.4f}')
print(f'LSTM Val Accuracy: {history["lstm_val_acc"]:.4f}')

## Step 5: Evaluate Model on Test Set

In [ ]:
metrics = model.evaluate(X_test_seq, y_test_seq)

print('\nModel Evaluation (Test Set):')
print(f'Accuracy: {metrics["accuracy"]:.4f}')
print(f'Precision: {metrics["precision"]:.4f}')
print(f'Recall: {metrics["recall"]:.4f}')
print(f'F1-Score: {metrics["f1"]:.4f}')
print(f'\nConfusion Matrix:')
print(metrics['confusion_matrix'])

## Step 6: Train Volatility Model

In [ ]:
vol_model = VolatilityModel(atr_multiplier=1.5, atr_window=14)

# Create labels and features
y_train_vol = vol_model.create_labels(train_df)
X_train_vol = vol_model.create_features(train_df)

y_val_vol = vol_model.create_labels(val_df)
X_val_vol = vol_model.create_features(val_df)

y_test_vol = vol_model.create_labels(test_df)
X_test_vol = vol_model.create_features(test_df)

# Train
vol_metrics = vol_model.train(X_train_vol, y_train_vol, X_val_vol, y_val_vol)

print('Volatility Model Training Metrics:')
for key, value in vol_metrics.items():
    print(f'  {key}: {value:.4f}')

# Evaluate on test set
vol_test_metrics = vol_model.evaluate(X_test_vol, y_test_vol)
print('\nVolatility Model Test Metrics:')
for key, value in vol_test_metrics.items():
    print(f'  {key}: {value:.4f}')

## Summary
Both signal classification and volatility models are now trained and evaluated!

Next steps:
1. Check if accuracy meets target (80%+)
2. If not, adjust hyperparameters in config.yaml
3. Run multi-symbol training
4. Deploy to production
